In [3]:
import os

In [4]:
%pwd

'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\assignment\\ETE_ML_Project\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\assignment\\ETE_ML_Project'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name : str
    alpha: float
    l1_ratio: float
    target_column: str
    scaler_name : str

In [8]:
from Mlproject.utils.common import *
from Mlproject.constants import *

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root]) 

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name,
            scaler_name = config.scaler_name
            
        )
        return model_trainer_config

In [13]:
import pandas as pd
import os
from Mlproject import logger
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from pathlib import Path

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        scale = StandardScaler()
        train_x = scale.fit_transform(train_x)
        test_x = scale.fit_transform(test_x)


        lr = LogisticRegression(penalty='elasticnet', 
                        solver='saga', 
                        C=1/self.config.alpha,  
                        l1_ratio=self.config.l1_ratio, 
                        random_state=42)
        lr.fit(train_x, train_y)

        


In [14]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-22 13:59:13,696: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-22 13:59:13,701: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-22 13:59:13,709: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-22 13:59:13,712: INFO: common: created directory at: artifacts]
[2024-08-22 13:59:13,723: INFO: common: created directory at: artifacts/model_training]


c:\Users\sredekar\OneDrive - Interpublic\Desktop\Datascience\assignment\ETE_ML_Project\pwvenv\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sredekar\OneDrive - Interpublic\Desktop\Datascience\assignment\ETE_ML_Project\pwvenv\lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'